# Loading and Exploring Gate-Level Circuits

Example of parsing the bench data format to make simple gate-level circuits.

In [1]:
from kyupy import bench

# parse a file
b01 = bench.parse('tests/b01.bench')

# ... or specify the circuit as string 
mycircuit = bench.parse('input(a,b) output(o1,o2,o3) x=buf(a) o1=not(x) o2=buf(x) o3=buf(x)')

0000000.334 W Cuda unavailable. Falling back to pure python


Circuits are objects of the class `Circuit`.

In [2]:
b01

<Circuit 'tests/b01' with 92 nodes, 130 lines, 4 ports>

In [3]:
mycircuit

<Circuit with 10 nodes, 8 lines, 5 ports>

Circuits are containers for two types of elements: nodes and lines.
* A `Node` is a named entity in a circuit (e.g. a gate, a standard cell, a named signal, or a fan-out point) that has connections to other nodes.
* A `Line` is a directional 1:1 connection between two Nodes.

Use the `dump()` method to get a string representation of all nodes and their connections.

In [4]:
print(mycircuit.dump())

None(0,1,2,3,4)
0:__fork__"a"  >1
1:__fork__"b"  
2:__fork__"o1" <2 
3:__fork__"o2" <4 
4:__fork__"o3" <6 
5:buf"x" <1 >0
6:__fork__"x" <0 >3 >5 >7
7:not"o1" <3 >2
8:buf"o2" <5 >4
9:buf"o3" <7 >6


The first line of the dump starts with the circuit name ("None" for `mycircuit`), followed by the node-IDs of all the ports (inputs and outputs) of the circuit.

Each of the following lines describes one node.
Each node in the circuit has a unique ID, a type, a name, and line-connections. This information is given on each line in that order.

A line in the circuit has a unique ID, a driver node and a receiver node. The connections in the dump show the direction (">" for output, "<" for input) and the line-ID. For example in `mycircuit`: Node-0 has one output connected to Line-1, and this Line-1 is connected to the input of Node-5.

The `interface` is the list of nodes forming the ports (inputs and outputs):

In [5]:
mycircuit.interface

[0:__fork__"a"  >1,
 1:__fork__"b"  ,
 2:__fork__"o1" <2 ,
 3:__fork__"o2" <4 ,
 4:__fork__"o3" <6 ]

## Nodes

There are two types of nodes: __forks__ and __cells__.

Forks have the special type `__fork__` while cells can be of various types (`buf`, `not`, `and`, `nor`, etc.).
Forks are used to label signals with names and to connect a one cell to multiple other cells (fan-out).
The names among all forks and among all cells within a circuit are unique.
Thus, a fork and a cell are allowed to share the same name.

Nodes in circuits can be accessed by ID or by name.

In [6]:
mycircuit.nodes[7]

7:not"o1" <3 >2

In [7]:
mycircuit.forks['x']

6:__fork__"x" <0 >3 >5 >7

In [8]:
mycircuit.cells['x']

5:buf"x" <1 >0

Nodes have an `index` (the node ID), a `kind` (the type), a `name`, as well as `ins` (input pins) and `outs` (output pins)

In [9]:
n = mycircuit.nodes[6]
n.index, n.kind, n.name, n.ins, n.outs

(6, '__fork__', 'x', [0], [3, 5, 7])

The inputs and outputs of a node are lists containing `Line` objects.

In [10]:
type(n.ins[0])

kyupy.circuit.Line

## Lines

A line is a directional connection between one driving node (`driver`) and one reading node (`reader`).

A line also knows to which node pins it is connected to: `driver_pin`, `reader_pin`.

In [11]:
l = mycircuit.nodes[6].outs[1]
l.index, l.driver, l.reader, l.driver_pin, l.reader_pin

(5, 6:__fork__"x" <0 >3 >5 >7, 8:buf"o2" <5 >4, 1, 0)

## Basic Analysis Examples
### Cell type statistics

In [12]:
from collections import defaultdict

counts = defaultdict(int)

for n in b01.cells.values():
    counts[n.kind] += 1

print(counts)

defaultdict(<class 'int'>, {'DFF': 5, 'AND': 1, 'NAND': 28, 'OR': 1, 'NOT': 10})


### Tracing a scan chain

In [13]:
from kyupy import verilog

b14 = verilog.parse('tests/b14.v.gz')
b14

<Circuit 'b14' with 15864 nodes, 23087 lines, 91 ports>

In [14]:
chain = []
cell = b14.cells['Scan_Out']
chain.append(cell)
while len(cell.ins) > 0:
    cell = cell.ins[2 if 'SDFF' in cell.kind else 0].driver
    if '__fork__' not in cell.kind:
        chain.append(cell)
        
print('chain length', len(chain))
print([c.name for c in chain])

chain length 229
['Scan_Out', 'u04_opt1329', 'u04_opt1328', 'wr_reg', 'u04_opt11', 'state_reg_0_0', 'reg3_reg_28_0', 'reg3_reg_27_0', 'reg3_reg_26_0', 'reg3_reg_25_0', 'reg3_reg_24_0', 'u04_opt1123', 'reg3_reg_23_0', 'reg3_reg_22_0', 'reg3_reg_21_0', 'u04_opt1118', 'reg3_reg_20_0', 'reg3_reg_19_0', 'reg3_reg_18_0', 'reg3_reg_17_0', 'reg3_reg_16_0', 'reg3_reg_15_0', 'reg3_reg_14_0', 'reg3_reg_13_0', 'reg3_reg_12_0', 'reg3_reg_11_0', 'reg3_reg_10_0', 'reg3_reg_9_0', 'reg3_reg_8_0', 'reg3_reg_7_0', 'reg3_reg_6_0', 'reg3_reg_5_0', 'reg3_reg_4_0', 'reg3_reg_3_0', 'reg3_reg_2_0', 'reg3_reg_1_0', 'reg3_reg_0_0', 'reg2_reg_31_0', 'reg2_reg_30_0', 'reg2_reg_29_0', 'reg2_reg_28_0', 'reg2_reg_27_0', 'reg2_reg_26_0', 'reg2_reg_25_0', 'reg2_reg_24_0', 'reg2_reg_23_0', 'reg2_reg_22_0', 'reg2_reg_21_0', 'reg2_reg_20_0', 'reg2_reg_19_0', 'reg2_reg_18_0', 'reg2_reg_17_0', 'reg2_reg_16_0', 'reg2_reg_15_0', 'reg2_reg_14_0', 'reg2_reg_13_0', 'reg2_reg_12_0', 'reg2_reg_11_0', 'reg2_reg_10_0', 'reg2_reg_9_0

# Loading SDFs and STILs

In [15]:
from kyupy import verilog, sdf
from kyupy.saed import pin_index
from kyupy import stil

b14 = verilog.parse('tests/b14.v.gz')
df = sdf.parse('tests/b14.sdf.gz')
lt = df.annotation(b14, pin_index, interconnect=False)
s = stil.parse('tests/b14.stil.gz')
t = s.tests8v(b14)

In [16]:
lt

array([[[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]],

       ...,

       [[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]]])

In [17]:
t[0]

'00-RFRF01F10FFRFF1FR1F1RR010F0F1RRR-------F------------------------------------------------11110110011100110111111110111000010000001111010111001111110110010101100100001000101001101010010011010000001111110111101110110001011010100011010001111010011101001000011111011101111101010111001100100011111100000101110'

## 32 Parallel Time Simulations with Waveform Capacity 16

This code will fall back to pure python if no CUDA card is available. This will be quite slow.

Instanciate simulator:

In [18]:
from kyupy.wave_sim_cuda import WaveSimCuda, TMAX
import numpy as np

wsim = WaveSimCuda(b14, lt, sims=32, wavecaps=16)

Main Simulation Loop

In [19]:
nvectors = 32 #len(t)
r = np.zeros((len(wsim.interface), nvectors, 1))

for offset in range(0, nvectors, wsim.sims):
    wsim.assign(t, offset=offset)
    wsim.propagate(sims=nvectors-offset)
    cdata = wsim.capture(time=TMAX, offset=offset)
    r = cdata[...,0]

Output some captures data

In [20]:
cdata.shape

(306, 32, 6)

In [21]:
r

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.]], dtype=float32)

### Check for CUDA Support

Try this code to check if CUDA is set up correctly.

In [ ]:
from numba import cuda

cuda.detect()